In [1]:
import math
import json
import numpy as np
import pandas as pd
import networkx as nx
#import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline
import os
import conda

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

from mpl_toolkits.basemap import Basemap

In [2]:
cities = pd.read_csv('data/Top50/top50cities.csv',delimiter=',')

In [3]:
#import nodes to get id from nodes/hosts
hosts = pd.DataFrame() #creates a new dataframe that's empty
for idx,city in cities.iterrows():#.loc[cities['2017']>50000].iterrows():
    for p in range(1,14):
        if (os.path.exists('data/Top50/hosts/'+city['name']+str(p)+'.csv')):
            df = pd.read_csv('data/Top50/hosts/'+city['name']+str(p)+'.csv',index_col=False)
            hosts = pd.concat([hosts,df]).reset_index(drop=True)
print(len(hosts))
hosts=hosts.drop_duplicates()
print(len(hosts))

10586
10582


In [4]:
rev = pd.DataFrame() #creates a new dataframe that's empty
for idx,city in cities.iterrows():#.loc[cities['2017']>50000].iterrows():
    if os.path.exists('data/Top50/reviews/'+city['name']+'Rev.csv'):
        df = pd.read_csv('data/Top50/reviews/'+city['name']+'Rev.csv',index_col=False)
        rev = pd.concat([rev,df]).reset_index(drop=True)
print(len(rev))
rev=rev.drop_duplicates()
print(len(rev))


96090
96085


In [5]:
!pip install geopy
from geopy import geocoders  


In [6]:
cities_not_found = []
def getCityCoords(name):
    # if geopy does not work -> http://www.geonames.org/login and log in with cred:
    # username: kacper
    # pass : zaq12345
    gn = geocoders.GeoNames(username='kacper')
    try:
        loc = gn.geocode(name, timeout=10)
        
        if loc is None:
            try: 
                loc =gn.geocode(name.split(',')[0],timeout=10)
                if loc is None:
                    raise TypeError
            except Exception as inst:
                print('This city was not found:',name)
                return None
    except Exception as inst:
        print('This city was not found:',name)
        cities_not_found.append(name)
        return None
    #print(len(loc))
   # print(loc[0].raw)
    return loc

In [7]:
allCities=rev.fromPublicAddressDescription.unique()
allCities = allCities.tolist()
allCities

['Setiabudi, Jakarta, Indonesia',
 'Dubai',
 'Seoul, Seoul, Korea (Republic of)',
 'Jihlava, Vysocina Region, Czechia',
 'Paihia, Northland, New Zealand',
 'Kirov Oblast, Russian Federation',
 'Hangzhou, xiasha, Zhejiang, China',
 'Seoul, Seoul, Korea, Republic of',
 'MD, USA',
 'Pereira, Risaralda, Colombia',
 'Djelfa Province, Algeria',
 'Barnaul, Altai Krai, Russian Federation',
 'Islamabad, Islamabad Capital Territory, Pakistan',
 'Kb. Jeruk, Daerah Khusus Ibukota Jakarta, Indonesia',
 'Riga, Riga, Latvia',
 'Saint Petersburg, Saint Petersburg, Russian Federation',
 'Bogota, Colombia',
 'Zürich, Zurich, Switzerland',
 'Bogotá, Bogota, Colombia',
 'Sankt-Peterburg, Saint Petersburg, Russian Federation',
 'Naberezhnye Chelny, Tatarstan, Russian Federation',
 'Moscow, Moscow, Russian Federation',
 'Ufa, Republic of Bashkortostan, Russian Federation',
 'Kyiv, Kyiv city, Ukraine',
 'Dakar, Dakar Region, Senegal',
 'Ivanovo, Ivanovo Oblast, Russian Federation',
 'Guilin, Guangxi, China',

In [9]:
#source: https://www.maxmind.com/en/free-world-cities-database
worldCities = pd.read_csv('../worldcitiespop.txt',encoding = "ISO-8859-1")
#source: https://datahub.io/core/country-list#resource-data
cc= pd.read_csv('data/country_map.txt')

/Users/Eike/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
worldCities.loc[worldCities.City=='hamburg']

,Country,City,AccentCity,Region,Population,Latitude,Longitude
744968,de,hamburg,Hamburg,4,1733846.0,53.550000,10.000000
2602816,se,hamburg,Hamburg,07,NaN,63.650000,12.883333
2658727,sr,hamburg,Hamburg,17,NaN,5.783333,-55.350000
2909343,us,hamburg,Hamburg,AL,NaN,32.527778,-87.286389
2915919,us,hamburg,Hamburg,AR,NaN,33.228056,-91.797500
2919866,us,hamburg,Hamburg,CA,NaN,41.783056,-123.059167
2925122,us,hamburg,Hamburg,CT,NaN,41.384444,-72.350556
2928647,us,hamburg,Hamburg,FL,NaN,30.565556,-83.526944
2939265,us,hamburg,Hamburg,IL,NaN,39.230000,-90.717500
2942648,us,hamburg,Hamburg,IN,NaN,38.383333,-85.767500


In [26]:
import unidecode

In [14]:
from googletrans import Translator
t = Translator()

In [171]:
s='Полярный'
#test=unidecode.unidecode(s)
#print(test.lower())
test=getCityCoords(s)
#worldCities.loc[worldCities.City==test.lower()]

In [172]:
print(test.latitude
test.address.split(' ')[0]

Polyarnyy Ural, 00, RU


'Polyarnyy'

In [186]:
rev['lat']='NaN'
rev['lng']='NaN'
rev['city']='NaN'
rev['country']='NaN'

In [199]:

exList=["nan"]
for city in allCities:
    
    if all([c[1]=='NaN'for c in rev.loc[rev.fromPublicAddressDescription==city]['lat'].iteritems()]):
        #print(city.split(',')[0].lower())

        if str(city).lower() not in exList:
            loc = worldCities.loc[worldCities.City==city.split(',')[0].lower().strip()]
            if len(loc.index)>1:
                #print('More Results',city)
                #print(loc)

                try:
                    countryCode = cc.loc[cc.Name==city.split(',')[-1].strip().lower().title()]['Code'].values[0]
                    #print('countryCode',countryCode)
                    loc =loc.loc[loc.Country==countryCode.lower()].head(1)
                    #print(loc)
                except:
                    loc=loc.head(1)
            if len(loc.head(1).index)==0:
                c=test=unidecode.unidecode(city)
                loc = worldCities.loc[worldCities.City==c.split(',')[0].lower().strip()].head(1)
                if len(loc.index)>1:
                    try:
                        countryCode = cc.loc[cc.Name==c.split(',')[-1].strip().lower().title()]['Code'].values[0]
                        loc =loc.loc[loc.Country==countryCode.lower()].head(1)
                    except:
                        loc=loc.head(1)  

            if len(loc.head(1).index)==1:
                l=rev.loc[rev.fromPublicAddressDescription==city].index.tolist()
                for i in l:
                    rev.set_value(i,'lat',loc.Latitude.values[0])
                    rev.set_value(i,'lng',loc.Longitude.values[0])
                    rev.set_value(i,'city',loc.City.values[0])
                    rev.set_value(i,'country',loc.Country.values[0])
                          
                #rev.loc[rev.fromPublicAddressDescription==city]['lat']=loc.Latitude
                #rev.loc[rev.fromPublicAddressDescription==city]['lng']=loc.Longitude
                #rev.loc[rev.fromPublicAddressDescription==city]['city']=loc.City
            
            else:
                print('try api for :',city)
                res = getCityCoords(city)
                try:
                    l=rev.loc[rev.fromPublicAddressDescription==city].index.tolist()
                    for i in l:
                        rev.set_value(i,'lat',res.latitude)
                        rev.set_value(i,'lng',res.longitude)
                        rev.set_value(i,'city',res.address.split(' ')[0].lower())
                        rev.set_value(i,'country',res.address.split(' ')[-1].lower())
                except:
                    print('Could not found ',city)
    

try api for : Bilhorod-Dnistrovs'kyi, Odessa Oblast, Ukraine
This city was not found: Bilhorod-Dnistrovs'kyi, Odessa Oblast, Ukraine
Could not found  Bilhorod-Dnistrovs'kyi, Odessa Oblast, Ukraine
try api for : Yenakijeve, Donetsk Oblast, Ukraine
This city was not found: Yenakijeve, Donetsk Oblast, Ukraine
Could not found  Yenakijeve, Donetsk Oblast, Ukraine
try api for : guayaqil, Ecuador
This city was not found: guayaqil, Ecuador
Could not found  guayaqil, Ecuador
try api for : La boulladisse, Provance Alpes, France
This city was not found: La boulladisse, Provance Alpes, France
Could not found  La boulladisse, Provance Alpes, France
try api for : Pervomais'k, Mykolaivs'ka oblast, Ukraine
This city was not found: Pervomais'k, Mykolaivs'ka oblast, Ukraine
Could not found  Pervomais'k, Mykolaivs'ka oblast, Ukraine
try api for : khajurajo, Madhya Pradesh, India
This city was not found: khajurajo, Madhya Pradesh, India
Could not found  khajurajo, Madhya Pradesh, India
try api for : Volgo

/Users/Eike/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/Eike/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/Eike/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/Eike/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


try api for : i was born in, Italy
This city was not found: i was born in, Italy
Could not found  i was born in, Italy
try api for : Plyos, Ivanovo Oblast. Привллжский район,, Russian Federation
This city was not found: Plyos, Ivanovo Oblast. Привллжский район,, Russian Federation
Could not found  Plyos, Ivanovo Oblast. Привллжский район,, Russian Federation
try api for : ChIJq5EYs65unUcRC-cFw9pQGh4
This city was not found: ChIJq5EYs65unUcRC-cFw9pQGh4
Could not found  ChIJq5EYs65unUcRC-cFw9pQGh4
try api for : Guishan dist, Taoyuan, Taiwan, Province of China
This city was not found: Guishan dist, Taoyuan, Taiwan, Province of China
Could not found  Guishan dist, Taoyuan, Taiwan, Province of China
try api for : Khmel'nyts'kyi, Khmel'nyts'ka oblast, Ukraine
This city was not found: Khmel'nyts'kyi, Khmel'nyts'ka oblast, Ukraine
Could not found  Khmel'nyts'kyi, Khmel'nyts'ka oblast, Ukraine
try api for : Olmalik, Uzbekistan
This city was not found: Olmalik, Uzbekistan
Could not found  Olmali

try api for : Pennyland, England, United Kingdom
This city was not found: Pennyland, England, United Kingdom
Could not found  Pennyland, England, United Kingdom
try api for : Ivano-Frankivs'ka oblast, Ukraine
try api for : Pen-y-fai, Cymru, United Kingdom of Great Britain and Northern Ireland
This city was not found: Pen-y-fai, Cymru, United Kingdom of Great Britain and Northern Ireland
Could not found  Pen-y-fai, Cymru, United Kingdom of Great Britain and Northern Ireland
try api for : State of Goiás, Brazil
This city was not found: State of Goiás, Brazil
Could not found  State of Goiás, Brazil
try api for : New Taipei City, Taiwan, Republic Of China
try api for : Heilingrabe, Prignitz, Germany
This city was not found: Heilingrabe, Prignitz, Germany
Could not found  Heilingrabe, Prignitz, Germany
try api for : Arabako Lautada, Basque Country, Spain
This city was not found: Arabako Lautada, Basque Country, Spain
Could not found  Arabako Lautada, Basque Country, Spain
try api for : Buch

/Users/Eike/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/Eike/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/Eike/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/Eike/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


...

try api for : Algueirão-Mem Martins, Lisboa, Portugal
try api for : Alchevs'k, Luhans'ka oblast, Ukraine
try api for : Planet Earth
This city was not found: Planet Earth
Could not found  Planet Earth
try api for : Upper Brighton, Saint George, Barbados
try api for : Nganmarriyanga, Northern Territory, Australia
This city was not found: Nganmarriyanga, Northern Territory, Australia
Could not found  Nganmarriyanga, Northern Territory, Australia
try api for : Korolyov, Moscow Province, Russia
try api for :  , Vitsyebsk, Vitebsk Region, BLR
try api for : Trigoria, Lazio, Italy
try api for : Cervia (Ravenna), Emilia-Romagna, Italy
try api for : Onokivtsi, Zakarpats'ka oblast, Ukraine
try api for : Tiachiv, Zakarpattia, Ukraine
try api for : Львов, Львовская область
try api for : Oaklands Park, South Australia, Australia
try api for : Senningerberg, District de Luxembourg, Luxembourg
try api for : jettigen-scheppach, Germany
This city was not found: jettigen-scheppach, Germany
Could not foun

This city was not found: Warszawa/Tychy/Cieszyn, Poland
Could not found  Warszawa/Tychy/Cieszyn, Poland
try api for : Goeppingen, Baden-Wuerttemberg, Germany
try api for : St Ives, Cornwall
try api for : Sint-Lambrechts-Woluwe - Woluwe-Saint-Lambert, Brussels, Belgium
try api for : Mengwi, Bali, Indonesia
try api for : Prior Velho, Lisboa, Portugal
try api for : Viewbank, Victoria, Australia
try api for : Piwniczna-Zdrój, Lesser Poland Voivodeship, Poland
try api for : Pyrgos Kallistis, Egeo, Greece
try api for : Karavomylos, Decentralized Administration of Peloponnese, Western Greece and the Ionian, Greece
try api for : Agualva-Cacém, Lisbon, Portugal
try api for : Nusa Penida, Bali, Indonesia
try api for : Valle del Guadalhorce, Andalusia, Spain
This city was not found: Valle del Guadalhorce, Andalusia, Spain
Could not found  Valle del Guadalhorce, Andalusia, Spain
try api for : Amasya Province, Turkey
try api for : Fislisbach, Aargau, Switzerland
try api for : Prétot-Sainte-Suzanne,

try api for : Drohobych, Lviv Oblast, Ukraine
try api for : Kearneys Spring, Queensland, Australia
try api for : Dzhalal-Abadskaya, Kyrgyzstan
try api for : Yin Chuan, Ning Xia, China
try api for : Pelekh, North District, Israel
try api for : Woolloongabba, Queensland, Australia
try api for : Schoeningen, Lower Saxony, Germany
try api for : Langweid am Lech, Bavaria, Germany
try api for : Bilambil Heights, New South Wales, Australia
try api for : San Germano dei Berici, Veneto, Italy
try api for : Schladmimg, Styria, Austria
This city was not found: Schladmimg, Styria, Austria
Could not found  Schladmimg, Styria, Austria
try api for : Taoyuan City, Taiwan, Republic Of China
try api for : Schwaebisch Hall, Baden-Wuerttemberg, Germany
try api for : Saint-Petersburg , Leningrad, Russia
try api for : Nova Ukrainka, Rivne Oblast, Ukraine
try api for : Upplands Vasby, Stockholm, Sweden
try api for : Magnetic Island, Queensland, Australia
try api for : Temuka, Timaru, New Zealand, New Zealand

try api for : Na'an, Central district, Israel
try api for : Whashington, D.C, USA
This city was not found: Whashington, D.C, USA
Could not found  Whashington, D.C, USA
try api for : Nova Cidade, São Gonçalo, Rio de Janeiro, Brazil
try api for : Montevideo Dept, Uruguay
This city was not found: Montevideo Dept, Uruguay
Could not found  Montevideo Dept, Uruguay
try api for : Oryol, Novgorodskaya oblast', Russian Federation
try api for : Konaklı Belediyesi, İzmir, Turkey
This city was not found: Konaklı Belediyesi, İzmir, Turkey
Could not found  Konaklı Belediyesi, İzmir, Turkey
try api for : Wadi Rum Village, Aqaba Governorate, Jordan
This city was not found: Wadi Rum Village, Aqaba Governorate, Jordan
Could not found  Wadi Rum Village, Aqaba Governorate, Jordan
try api for : Днепр, +380
try api for : Kempten (Allgäu), Bayern, Germany
try api for : La Paz Department, Bolivia (Plurinational State of)
try api for : Baztan, Navarre (Community of), Spain
try api for : Mollymook, New South Wa

This city was not found: Yas'ur, North District, Israel
Could not found  Yas'ur, North District, Israel
try api for : Saint-Petersberg, Russian Federation
try api for : Bondi Beach, New South Wales, Australia
try api for : Frascati (Rm), Lazio, Italy
try api for : Pachuca (Mineral de la reforma), Hidalgo, Mexico
try api for : Ste Rose Du Lac, Manitoba, Canada
try api for : katerina_kavecka@inbox.lv
This city was not found: katerina_kavecka@inbox.lv
Could not found  katerina_kavecka@inbox.lv
try api for : Baden/Vienna, Austria
try api for : Nago-torbole, Trentino-Alto Adige, Italy
try api for : Les Moutiers en Retz, Pays de la Loire, France
try api for : Skåne County, Sweden
try api for : Altdorf bei Nürnberg, Bavaria, Germany
try api for : Tainan City, Taiwan
try api for : Beit El Chaar, Jabal Lubnan, Lebanon
This city was not found: Beit El Chaar, Jabal Lubnan, Lebanon
Could not found  Beit El Chaar, Jabal Lubnan, Lebanon
try api for : ChIJi73bYWusQjQRgqQGXK260bw
This city was not fou

In [200]:
export_csv = rev.to_csv ('data/reviews_total_geo_2.csv', index = None, header=True)

In [90]:
G = nx.MultiDiGraph()
links = []
user_id_unique=set(hosts['id'].tolist())
G.add_nodes_from([(rows['name'],{'lat':rows['lat'],'lng':rows['lng']}) for idx,rows in cities.iterrows()])
G.add_nodes_from([(rows['fromPublicAddressDescription'],{'lat':rows['lat'],'lng':rows['lng']}) for idx,rows in rev.iterrows() if rows['lat'] !='NaN' and rows['lng']!='NaN'])
#for idx,r in rev.iterrows():
    #if r['from'] in hosts['id'].tolist() and r['to'] in hosts['id'].tolist():
    #if r['from'] in list(G.nodes()):
    #    if r['relationshipType']=='surf':
    #        #G.add_edge(hosts.loc[hosts['id']==r['from']]['city'].item(),hosts.loc[hosts['id']==r['to']]['city'].item())
    #        G.add_edge()
    #    if r['relationshipType']=='host':
    #        G.add_edge(hosts.loc[hosts['id']==r['to']]['city'].item(),hosts.loc[hosts['id']==r['from']]['city'].item())
#G.add_edges_from([(rows['to'],rows['from'])for index,rows in rev[['to','from','relationshipType']].iterrows() if rows['relationshipType']=='host' and G.has_node(rows['to']) and G.has_node(rows['from'])])
#G.add_edges_from([(rows['from'],rows['to'])for index,rows in rev[['to','from','relationshipType']].iterrows() if rows['relationshipType']=='surf' and G.has_node(rows['to']) and G.has_node(rows['from'])])



In [196]:
for c in rev.loc[rev.fromPublicAddressDescription=='Setiabudi, Jakarta, Indonesia']['lat'].iteritems():
    print(c[1]!=)

3.695


In [152]:
test=worldCities.loc[worldCities.City=='pereira']
test.loc[test.Country=='co'].head(1)

,Country,City,AccentCity,Region,Population,Latitude,Longitude
679302,co,pereira,Pereira,12,NaN,8.516569,-76.077564


In [112]:
cc.loc[cc.Name=='dubai'.title()]['Code']


Series([], Name: Code, dtype: object)

In [189]:
rev[['fromPublicAddressDescription','city','lat','lng','country']]

,fromPublicAddressDescription,city,lat,lng,country
0,"Setiabudi, Jakarta, Indonesia",setiabudi,3.695,96.868,id
1,Dubai,dubai,25.2582,55.3047,ae
2,"Seoul, Seoul, Korea (Republic of)",seoul,37.5985,126.978,kr
3,"Jihlava, Vysocina Region, Czechia",jihlava,49.4,15.5833,cz
4,"Paihia, Northland, New Zealand",paihia,-35.2833,174.083,nz
5,"Kirov Oblast, Russian Federation","kirov,",58.5966,49.6601,ru
6,"Hangzhou, xiasha, Zhejiang, China",hangzhou,30.2936,120.161,cn
7,"Seoul, Seoul, Korea, Republic of",seoul,37.5985,126.978,kr
8,"MD, USA","baltimore-towson,",39.2881,-76.6108,us
9,"Pereira, Risaralda, Colombia",pereira,8.51657,-76.0776,co


In [201]:
rev2= pd.read_csv('data/reviews_total_geo_2.csv')
rev2

/Users/Eike/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,7,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,text,createdDate,experience,relationshipType,isPostTrip,to,from,fromPublicName,fromPublicAddressId,...,fromStatus,fromIsVerified,fromIsDeleted,fromBlockedBy,inverseReference,response,lat,lng,city,country
0,27077307,Amazing chef and humble human being. Willing t...,2018-10-17T22:09:22Z,positive,host,True,2009059021,2001999679,Dhanush Anand,109610804.0,...,yes,True,False,NaN,NaN,NaN,3.695000,96.868000,setiabudi,id
1,27152728,Amazing boy with amazing family . He met me up...,2018-10-28T19:16:31Z,positive,surf,True,2009059021,2007724821,Bakhar Sardzhaeva,271153483.0,...,no,False,False,NaN,NaN,NaN,25.258172,55.304717,dubai,ae
2,26922175,￼\n\n \n\nWe use cookies to enhance your exper...,2018-09-27T14:41:06Z,positive,surf,True,2008295707,2008797517,Sun-hyung Kim,312790604.0,...,maybe,False,False,NaN,NaN,NaN,37.598500,126.978300,seoul,kr
3,26866500,"I was ill, when I come to Alexey's place, so I...",2018-09-20T17:32:11Z,positive,surf,True,2008295707,2003041472,Michelle Kuglerová,123277124.0,...,maybe,False,False,NaN,NaN,NaN,49.400000,15.583333,jihlava,cz
4,26818885,Didn't end up staying at his house because of....,2018-09-15T05:17:31Z,positive,surf,True,2008295707,4294146,Yvette Shaw,6999267.0,...,no,False,False,NaN,NaN,NaN,-35.283333,174.083333,paihia,nz
5,26779351,С Алексеем было очень приятно и комфортно пров...,2018-09-10T12:56:37Z,positive,surf,True,2008295707,2008843836,Angelika Krasnozhonova,314663738.0,...,hang,True,False,NaN,NaN,NaN,58.596650,49.660070,"kirov,",ru
6,26176442,I had a great experience in Russia with Alexei...,2018-07-10T13:57:26Z,positive,surf,True,2008295707,2007359140,Diego Gabriel Cortez Fernandez,250795609.0,...,no,False,False,NaN,NaN,NaN,30.293650,120.161419,hangzhou,cn
7,26164016,I bet Alexey is definitely a good person. He l...,2018-07-09T09:52:29Z,positive,surf,True,2008295707,2007634207,Heejeong Yang,265596282.0,...,no,False,False,NaN,NaN,NaN,37.598500,126.978300,seoul,kr
8,26148751,My experience with Alexey was awesome. He is ...,2018-07-07T16:05:53Z,positive,surf,True,2008295707,2005150941,Siboney Godoy,176314139.0,...,hang,False,False,NaN,NaN,NaN,39.290380,-76.612190,"baltimore,",us
9,26119628,Alexey is a really amazing host!! At the first...,2018-07-04T09:15:01Z,positive,surf,True,2008295707,1003812034,Emma Jimenez,10585071.0,...,maybe,False,False,NaN,NaN,NaN,8.516569,-76.077564,pereira,co


In [215]:
test=pd.to_numeric(rev2['lat'], errors='coerce').isnull().tolist()

In [216]:
sum(test)

815

In [217]:
815/96085

0.008482073164385701